In [38]:
import os
import numpy as np
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,GlobalMaxPooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [39]:
train_directory = r"c:\Users\Mohamed Arshad\Desktop\cell_images\train"
test_directory = r"C:\Users\Mohamed Arshad\Desktop\cell_images\test"

##### Data Augementation:
It will generate flips,upside down,zoom etc to learn model more efficiently

In [40]:
#Resnet50 expects image in 224*224 dimensions similiar to vgg16,vgg19
img_width,img_height=224,224

preprocess_for_resnet=preprocess_input

train_datagen=ImageDataGenerator(preprocessing_function=preprocess_for_resnet,
                                 zoom_range=0.2,
                                 shear_range=0.2,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 horizontal_flip=True)

test_datagen=ImageDataGenerator(preprocessing_function=preprocess_for_resnet)

train_data=train_datagen.flow_from_directory(train_directory,
                                             target_size=(img_width,img_height),
                                             batch_size=32,
                                             class_mode='binary')

test_data=test_datagen.flow_from_directory(test_directory,
                                             target_size=(img_width,img_height),
                                             batch_size=32,
                                             class_mode='binary')



Found 22046 images belonging to 2 classes.
Found 5512 images belonging to 2 classes.


Load ResNet50-Pretrained Transfer Learning CNN Model



In [41]:
resnet_model=ResNet50(weights='imagenet',include_top=False,input_shape=(img_width,img_height,3))

x=resnet_model.output
x=GlobalMaxPooling2D()(x)
x=Dense(units=128,activation='relu')(x)
final_layer=Dense(units=1,activation="sigmoid")(x)

model=Model(inputs=resnet_model.input,outputs=final_layer)

for layer in resnet_model.layers:
    layer.trainable=False

Compile and Run the Model

In [42]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_8[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [43]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['Accuracy'])
model.fit(x=train_data,validation_data=test_data,epochs=5)



Epoch 1/5
689/689 [==============================] - 750s 1s/step - loss: 0.4025 - Accuracy: 0.8982 - val_loss: 0.1528 - val_Accuracy: 0.9452
Epoch 2/5
689/689 [==============================] - 783s 1s/step - loss: 0.2184 - Accuracy: 0.9223 - val_loss: 0.1482 - val_Accuracy: 0.9456
Epoch 3/5
689/689 [==============================] - 821s 1s/step - loss: 0.1876 - Accuracy: 0.9339 - val_loss: 0.1643 - val_Accuracy: 0.9367
Epoch 4/5
689/689 [==============================] - 838s 1s/step - loss: 0.1811 - Accuracy: 0.9347 - val_loss: 0.1457 - val_Accuracy: 0.9447
Epoch 5/5
689/689 [==============================] - 777s 1s/step - loss: 0.1748 - Accuracy: 0.9365 - val_loss: 0.1469 - val_Accuracy: 0.9476


In [46]:
model.save("Trained_Malaria.model")

INFO:tensorflow:Assets written to: Trained_Malaria.model\assets


INFO:tensorflow:Assets written to: Trained_Malaria.model\assets
